In [1]:
import os
import pandas as pd
import numpy as np
import glob
import datetime

dataPathLs = np.array(glob.glob(r'A:\temp\full_20201228_zs_94_03_day_staserver\staMd***'))
mbd = []
for i in dataPathLs:
    mbd += [pd.read_csv(i)]
mbd = pd.concat(mbd)
dataPathLs = np.array(glob.glob(r'A:\temp\full_20201228_zs_94_03_day_staserver\mdOrderLog***'))
order = []
for i in dataPathLs:
    order += [pd.read_csv(i)]
order = pd.concat(order)
dataPathLs = np.array(glob.glob(r'A:\temp\full_20201228_zs_94_03_day_staserver\mdTradeLog***'))
trade = []
for i in dataPathLs:
    trade += [pd.read_csv(i)]
trade = pd.concat(trade)

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
thisDate = '20201228'
readPath = '\\\\192.168.10.28\\equityTradeLogs'
rawOrderLog = pd.read_csv(os.path.join(readPath, 'speedCompare_%s.csv'%thisDate))
for col in ['clockAtArrival', 'caamd', 'secid', 'updateType', 'vai', 'absFilledThisUpdate', 'orderDirection', 'absOrderSize',
            'absOrderSizeCumFilled', 'date', 'accCode', 'mse']:
    rawOrderLog[col] = rawOrderLog[col].fillna(0)
    rawOrderLog[col] = rawOrderLog[col].astype('int64')  
rawOrderLog = rawOrderLog.sort_values(by=['date', 'secid', 'vai', 'accCode', 'clockAtArrival']).reset_index(drop=True)

rawOrderLog = rawOrderLog[rawOrderLog["secid"] >= 1000000]

display('There are accounts with duplicated ticks:')
display(rawOrderLog[rawOrderLog.duplicated(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep=False)]\
.groupby(['date', 'colo', 'accCode'])['ars'].size())
rawOrderLog = rawOrderLog.drop_duplicates(['date', 'secid', 'vai', 'accCode', 'clockAtArrival', 'updateType', \
                                    'orderDirection', 'absOrderSize'], keep='first')

display('There are ticks with orderDirection 0')
display(rawOrderLog[rawOrderLog['orderDirection'] == 0][['date', 'colo', 'accCode', \
            'secid', 'vai', 'updateType', 'sdd', 'orderDirection', 'absOrderSize', 'internalId', 'orderId']])

assert(rawOrderLog[rawOrderLog['updateType'] == 0][rawOrderLog[rawOrderLog['updateType'] == 0]\
                                                   .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                'vai', 'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
try:
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)].shape[0] == 0)
except:
    print('There are orders with all things same except sdd')
    print(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId'], keep=False)])
    assert(rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)][rawOrderLog[(rawOrderLog['updateType'] == 0) & (rawOrderLog['accCode'] != 8856)]\
                                                       .duplicated(['date', 'colo', 'accCode', 'secid', 'orderDirection',
                                                                    'absOrderSize', 'internalId', 'sdd'], keep=False)].shape[0] == 0)
try:
    assert(sum(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() != 1) == 0) 
except:
    print('There are orders with same internalId but different orderId other than accCode 8856 case')
    print(rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique()[rawOrderLog[(rawOrderLog['updateType'] != 0) & (rawOrderLog['accCode'] != 8856)].groupby(['date', 'colo', 'accCode', 'secid', 
                'orderDirection', 'absOrderSize', 'internalId'])['orderId'].nunique() > 1])

r2 = rawOrderLog[(rawOrderLog['accCode'] != 8856) & (rawOrderLog['orderDirection'] != 0)]
r1 = rawOrderLog[(rawOrderLog['accCode'] == 8856) & (rawOrderLog['orderDirection'] != 0)]
r1['test'] = r1.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize']).grouper.group_info[0]
r1 = r1.sort_values(by=['test', 'clockAtArrival'])
r1.loc[r1['updateType'] != 0, 'vai'] = np.nan
r1['vai'] = r1.groupby('test')['vai'].ffill()
r2['test'] = r2.groupby(['date', 'colo', 'accCode', 'secid', 
            'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
r2 = r2.sort_values(by=['test', 'clockAtArrival'])
r2.loc[r2['updateType'] != 0, 'vai'] = np.nan
r2['vai'] = r2.groupby('test')['vai'].ffill()
assert(sum(r1[r1['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
try:
    assert(sum(r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1) == 0)
except:
    print('There are orders in 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    a = r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique()[r2[r2['updateType'] != 0].groupby(['test', 'vai'])['orderId'].nunique() != 1].reset_index()
    print(pd.merge(r2, a[['test', 'vai']], on=['test', 'vai'], how='inner')[['secid', 'accCode', 'colo', 'vai', 'updateType', 'sdd', 'internalId', 'orderId', 'absOrderSize', 'absFilledThisUpdate', 'absOrderSizeCumFilled', 'orderPrice', 'tradePrice']])
rawOrderLog = pd.concat([r1, r2])
del r1
del r2  


rawOrderLog = rawOrderLog.sort_values(by=['date', 'colo', 'accCode', 'secid', 'vai', 'clockAtArrival']).reset_index(drop=True)
rawOrderLog['clock'] = rawOrderLog['clockAtArrival'].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
rawOrderLog["broker"] = np.where(rawOrderLog["accCode"].astype(str).apply(lambda x: len(x) == 6), rawOrderLog['accCode'] // 10000, rawOrderLog['accCode'] // 100)
rawOrderLog['colo_broker'] = rawOrderLog['colo'].str[:2] + '_' + rawOrderLog['broker'].astype('str')
rawOrderLog['colo_account'] = rawOrderLog['colo'].str[:2] + '_' + rawOrderLog['accCode'].astype('str')
rawOrderLog['order'] = rawOrderLog.groupby(['date', 'colo', 'accCode', 'secid', 'vai', 'orderDirection', 'absOrderSize', 'internalId']).grouper.group_info[0]
rawOrderLog['group'] = rawOrderLog.groupby(['date', 'secid', 'vai']).grouper.group_info[0]
rawOrderLog['startClock'] = rawOrderLog.groupby(['order'])['caamd'].transform('first')
rawOrderLog['duration'] = rawOrderLog['clockAtArrival'] - rawOrderLog['startClock']
rawOrderLog['orderPrice'] = rawOrderLog['orderPrice'].apply(lambda x: round(x, 2))
rawOrderLog['tradePrice'] = rawOrderLog['tradePrice'].apply(lambda x: round(x, 2))
rawOrderLog['orderDirection1'] = np.where(rawOrderLog["orderDirection"] == -2, -1, np.where(
    rawOrderLog["orderDirection"] == 2, 1, rawOrderLog["orderDirection"]))
rawOrderLog['sdd'] = rawOrderLog.groupby('order')['sdd'].transform('first')
rawOrderLog['caamd'] = rawOrderLog.groupby('order')['caamd'].transform('first')
rawOrderLog["ars"] = rawOrderLog.groupby(['order'])['ars'].transform('first')
orderLog = rawOrderLog.copy()

orderLog['exchange'] = np.where(orderLog['secid'] >= 2000000, 'SZE', 'SSE')
orderLog['orderNtl'] = orderLog['orderPrice'] * orderLog['absOrderSize']
orderLog['tradeNtl'] = np.where(orderLog['updateType'] == 4, orderLog['tradePrice']*orderLog['absFilledThisUpdate'], 0)
orderLog['isMsg'] = np.where(orderLog['updateType'] == 0, 
                             np.where(orderLog['mse'] == 100, 1, 0), np.nan)
orderLog['isMsg'] = orderLog.groupby(['order'])['isMsg'].ffill()
orderLog['firstUpdateType'] = orderLog.groupby(['order'])['updateType'].transform('first')

print('1. delete orders firstUpdateType != 0')
print(orderLog[orderLog['firstUpdateType'] != 0]['order'].nunique())

orderLog = orderLog[orderLog['firstUpdateType'] == 0]
orderLog['insertNum'] = np.where(orderLog['updateType'] == 0, 1, 0)
orderLog['insertNum'] = orderLog.groupby(['order'])['insertNum'].transform('sum')

print('2. delete orders more than 2 updateType == 0')
print(orderLog[orderLog['insertNum'] != 1]['order'].nunique())

orderLog = orderLog[orderLog['insertNum'] == 1]
orderLog = orderLog[orderLog['colo'] == 'zs_94_03']

C:\Users\win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (27,55,64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


'There are accounts with duplicated ticks:'

Series([], Name: ars, dtype: int64)

'There are ticks with orderDirection 0'

,date,colo,accCode,secid,vai,updateType,sdd,orderDirection,absOrderSize,internalId,orderId
314035,20201228,zs_66_01,6634,2000007,-1,1,49597.0,0,0,-1.0,1.812811e+10
342328,20201228,zs_52_09,5291,2000553,-1,1,53524.0,0,0,-1.0,1.854590e+06
347596,20201228,zs_54_01,5474,2000608,-1,1,47253.0,0,0,-1.0,8.689127e+08
349238,20201228,zs_54_01,5474,2000619,-1,7,47362.0,0,0,-1.0,8.689128e+08
423299,20201228,zs_96_08,9685,2002100,-1,1,48345.0,0,0,-1.0,7.221449e+17
424530,20201228,zs_92_02,9243,2002108,-1,1,40760.0,0,0,-1.0,8.250502e+09
456478,20201228,zs_96_08,9685,2002274,-1,1,53632.0,0,0,-1.0,7.221449e+17
483038,20201228,zs_52_09,5264,2002410,-1,1,34355.0,0,0,-1.0,9.338200e+04
517378,20201228,zs_54_01,5474,2002598,-1,1,53196.0,0,0,-1.0,8.689184e+08
554629,20201228,zs_54_01,5474,2002765,-1,1,53781.0,0,0,-1.0,8.689190e+08


There are orders with same internalId but different orderId other than accCode 8856 case
date      colo      accCode  secid    orderDirection  absOrderSize  internalId
20201228  zs_54_01  5456     2300036  -1              1000          2093.0        2
Name: orderId, dtype: int64


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


There are orders in 8856 with same internalId and various orderId!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
     secid  accCode      colo        vai  updateType          sdd  internalId  \
0  2300036     5456  zs_54_01  2652411.0           0  112946610.0      2093.0   
1  2300036     5456  zs_54_01  2652411.0           2         -1.0      2093.0   
2  2300036     5456  zs_54_01  2652411.0           1      46800.0      2093.0   
3  2300036     5456  zs_54_01  2652411.0           3         -1.0      2093.0   

       orderId  absOrderSize  absFilledThisUpdate  absOrderSizeCumFilled  \
0         -1.0          1000                    0                      0   
1  868912271.0          1000                    0                      0   
2       2093.0          1000                    0                      0   
3       2093.0          1000                    0                      0   

   orderPrice  tradePrice  
0       17.63        -1.0  
1       17.63        -1.0  
2       17.63        -1.0  
3  

In [3]:
targetStock = orderLog['secid'].unique()
targetStock = np.array([int(str(i)[1:]) for i in targetStock])
targetStockSZ = sorted(targetStock[targetStock < 600000])
order = order[order['SecurityID'].isin(targetStockSZ)]
order['OrderType'] = order['OrderType'].astype(str)
order = order[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'Side',
                     'OrderType', 'Price', 'OrderQty']]

trade['ExecType'] = trade['ExecType'].astype(str)
trade = trade[trade['SecurityID'].isin(targetStockSZ)]
trade['volumeThisUpdate'] = np.where(trade['ExecType'] == 'F', trade['TradeQty'], 0)
trade['cum_volume'] = trade.groupby(['SecurityID'])['volumeThisUpdate'].cumsum()
trade = trade[['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'ApplSeqNum', 'cum_volume',
                     'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum', 'OfferApplSeqNum']]

rawMsgDataSZ = pd.concat([order, trade], sort=False)
del order
# del trade

rawMsgDataSZ = rawMsgDataSZ.sort_values(by=['sequenceNo']).reset_index(drop=True)

rawMsgDataSZ['cum_volume'] = rawMsgDataSZ.groupby(['SecurityID'])['cum_volume'].ffill().bfill()
rawMsgDataSZ['ExecType'] = rawMsgDataSZ['ExecType'].fillna('2')
rawMsgDataSZ['TradeQty'] = rawMsgDataSZ['TradeQty'].fillna(0)

saveCols = ['clockAtArrival', 'sequenceNo', 'TransactTime', 'SecurityID', 'cum_volume', 'ApplSeqNum', 
        'Side', 'OrderType', 'Price', 'OrderQty', 'ExecType', 'TradePrice', 'TradeQty', 'TradeMoney',
        'BidApplSeqNum', 'OfferApplSeqNum']
rawMsgDataSZ = rawMsgDataSZ[saveCols]

orderDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '2'][['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'OrderQty', 'Price', 'cum_volume', "TransactTime"]].reset_index(drop=True)
orderDataSZ['updateType'] = 0
tradeDataSZ = pd.concat([rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'BidApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]],
                         rawMsgDataSZ[rawMsgDataSZ['ExecType'] == 'F'][['SecurityID', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]], sort=False)
tradeDataSZ['ApplSeqNum'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), tradeDataSZ['OfferApplSeqNum'], tradeDataSZ['BidApplSeqNum'])
tradeDataSZ['Side'] = np.where(tradeDataSZ['BidApplSeqNum'].isnull(), 2, 1)
tradeDataSZ = tradeDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]]
tradeDataSZ['updateType'] = 4
cancelDataSZ = rawMsgDataSZ[rawMsgDataSZ['ExecType'] == '4'][['SecurityID', 'BidApplSeqNum', 'OfferApplSeqNum', 'clockAtArrival', 'sequenceNo', 'TradePrice', 'TradeQty', 'cum_volume', "TransactTime"]].reset_index(drop=True)
cancelDataSZ['ApplSeqNum'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, cancelDataSZ['OfferApplSeqNum'], cancelDataSZ['BidApplSeqNum'])
cancelDataSZ['Side'] = np.where(cancelDataSZ['BidApplSeqNum'] == 0, 2, 1)
cancelDataSZ = cancelDataSZ[['SecurityID', 'ApplSeqNum', 'clockAtArrival', 'sequenceNo', 'Side', 'TradeQty', 'cum_volume', "TransactTime"]]
cancelDataSZ['updateType'] = 3

msgDataSZ = pd.concat([orderDataSZ, tradeDataSZ, cancelDataSZ], sort=False)
del orderDataSZ
del tradeDataSZ
del cancelDataSZ

msgDataSZ = msgDataSZ.sort_values(by=['SecurityID', 'ApplSeqNum', 'sequenceNo']).reset_index(drop=True)
msgDataSZ['TradePrice'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradePrice'], 0)
msgDataSZ['TradePrice'] = msgDataSZ['TradePrice'].astype('int64')
msgDataSZ['TradeQty'] = np.where(msgDataSZ['updateType'] == 4, msgDataSZ['TradeQty'], 0)
msgDataSZ['TradeQty'] = msgDataSZ['TradeQty'].astype('int64')
msgDataSZ['secid'] = msgDataSZ['SecurityID'] + 2000000
assert(msgDataSZ['ApplSeqNum'].max() < 1e8)
msgDataSZ['StockSeqNum'] = msgDataSZ['SecurityID']*1e8 + msgDataSZ['ApplSeqNum']
msgDataSZ['date'] = int(thisDate) 
print('finish market orders')


# 2. orderLog
infoData = orderLog[(orderLog["isMsg"] == 1) & (orderLog['updateType'].isin([0, 3, 4]))].reset_index(drop=True)
infoData['Price'] = infoData['orderPrice'].apply(lambda x: round(x*100, 0))
infoData['Price'] = infoData['Price'].astype('int64')*100
infoData['OrderQty'] = infoData['absOrderSize']
infoData['Side'] = np.where(infoData['orderDirection1'] == 1, 1, 2)
infoData['TradePrice'] = np.where(infoData['updateType'] == 4, round(infoData['tradePrice']*100, 0), 0)
infoData['TradePrice'] = infoData['TradePrice'].astype('int64')*100
statusInfo = infoData.groupby(['order'])['updateType'].apply(lambda x: tuple(x)).reset_index()
statusInfo.columns = ['order', 'statusLs']
tradePriceInfo = infoData.groupby(['order'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
tradePriceInfo.columns = ['order', 'TradePriceLs']
tradeQtyInfo = infoData.groupby(['order'])['absFilledThisUpdate'].apply(lambda x: tuple(x)).reset_index()
tradeQtyInfo.columns = ['order', 'TradeQtyLs']
infoData = infoData[infoData['updateType'] == 0]
infoData = pd.merge(infoData, statusInfo, how='left', on=['order'], validate='one_to_one')
infoData = pd.merge(infoData, tradePriceInfo, how='left', on=['order'], validate='one_to_one')
infoData = pd.merge(infoData, tradeQtyInfo, how='left', on=['order'], validate='one_to_one')
infoData['brokerNum'] = infoData.groupby(['date', 'secid', 'vai', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'])['colo_account'].transform('count')
display(infoData[infoData['brokerNum'] >= 2].groupby(['colo', 'accCode'])['date'].count())
display('%.2f%%'%(infoData[infoData['brokerNum'] >= 2].shape[0] / infoData.shape[0]*100))
display(infoData[infoData['brokerNum'] >= 2].shape[0])
display(infoData.shape[0])
infoData = infoData[infoData['brokerNum'] == 1]
infoData = infoData[['date', 'secid', 'vai', 'ars', "mrstaat", "mrstauc", 'exchange', 'group', 'Price', 'OrderQty', 
                     'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum', 'order', 'colo', 'accCode', 
                     'clockAtArrival', 'orderDirection', 'absOrderSize']]   

print('finish our orders')

# 3. find the position of our orders
checkLog = msgDataSZ[msgDataSZ['updateType'] == 0]

checkLog = pd.merge(checkLog, infoData.drop_duplicates(subset=['secid', 'ApplSeqNum'])[['secid', 'ApplSeqNum', 'ars']], 
                on=['secid', 'ApplSeqNum'], how='left')
checkLog['ApplSeqNum'] = np.where(~checkLog['ars'].isnull(), checkLog['ApplSeqNum'], checkLog['ars'])
checkLog['start_time'] = np.where(~checkLog['ars'].isnull(), checkLog['clockAtArrival'], checkLog['ars'])
checkLog.drop(["ars"],axis=1,inplace=True)
checkLog['ApplSeqNum'] = checkLog.groupby(['secid'])['ApplSeqNum'].ffill()
checkLog['start_time'] = checkLog.groupby(['secid'])['start_time'].ffill()
checkLog['end_time'] = checkLog['start_time'] + 100*1e3
checkLog = checkLog[(~checkLog['ApplSeqNum'].isnull()) & (checkLog['clockAtArrival'] > checkLog['start_time']) & 
                   (checkLog['clockAtArrival'] < checkLog['end_time'])]
message = msgDataSZ[['StockSeqNum', 'clockAtArrival', 'updateType', 'sequenceNo', 'secid', 'ApplSeqNum', 'date']].copy()
msgDataSZ = msgDataSZ[msgDataSZ['StockSeqNum'].isin(checkLog['StockSeqNum'].values)]
print('finish get the interval')

def getTuple(x):
    return tuple(i for i in x)
    
statusInfo = msgDataSZ.groupby(['StockSeqNum'])['updateType'].apply(lambda x: getTuple(x)).reset_index()
statusInfo.columns = ['StockSeqNum', 'statusLs']
tradePriceInfo = msgDataSZ.groupby(['StockSeqNum'])['TradePrice'].apply(lambda x: tuple(x)).reset_index()
tradePriceInfo.columns = ['StockSeqNum', 'TradePriceLs']
tradeQtyInfo = msgDataSZ.groupby(['StockSeqNum'])['TradeQty'].apply(lambda x: tuple(x)).reset_index()
tradeQtyInfo.columns = ['StockSeqNum', 'TradeQtyLs']
del msgDataSZ

checkLog = pd.merge(checkLog, statusInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
checkLog = pd.merge(checkLog, tradePriceInfo, how='left', on=['StockSeqNum'], validate='one_to_one')
checkLog = pd.merge(checkLog, tradeQtyInfo, how='left', on=['StockSeqNum'], validate='one_to_one')  
del statusInfo
del tradePriceInfo
del tradeQtyInfo
checkLog = checkLog.rename(columns={'clockAtArrival':'caa_orderLog'})

try:
    checkLog = pd.merge(checkLog, infoData, how='left', on=['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], validate='many_to_one')
except:
    print('There are orders with same pattern but different vai, same ApplSeqNum')
    display([infoData[infoData.duplicated(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)]])
    checkLog = pd.merge(checkLog, infoData, how='left', on=['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'])
del infoData

checkLog = checkLog.drop_duplicates(['date', 'secid', 'Price', 'OrderQty', 'Side', 'statusLs', 'TradePriceLs', 'TradeQtyLs', 'ApplSeqNum'], keep=False)
checkLog = checkLog[~checkLog['accCode'].isnull()]
checkLog = checkLog.reset_index(drop=True)

finish market orders


Series([], Name: date, dtype: int64)

'0.00%'

0

1322

finish our orders
finish get the interval


In [4]:
test = orderLog[orderLog['updateType'].isin([0, 4, 3])]
display(checkLog.shape[0])
test = pd.merge(checkLog[['date', 'secid', 'order', 'start_time', 'accCode', 'StockSeqNum']], test[['updateType', 'date', 'secid', 'order', 'accCode', 'clockAtArrival', 'caamd', 'ApplSeqNum']], on=['date', 'secid', 'order', 'accCode'], how='inner')
display(test[test['updateType'] == 0].shape[0])
test = test.sort_values(by=['order', 'clockAtArrival'])
message = message.sort_values(by=['StockSeqNum', 'clockAtArrival'])
test['innerSeq'] = test.groupby('order')['date'].cumcount()
message['innerSeq'] = message.groupby('StockSeqNum')['date'].cumcount()
test = pd.merge(message, test, on=['StockSeqNum', 'updateType', 'innerSeq'], how='inner')
display(test[test['updateType'] == 0].shape[0])
test = test[test.columns[test.columns != 'secid_y']]
test = test[test.columns[test.columns != 'date_y']]
test = test.rename(columns={'clockAtArrival_x':'caa_raw', 'clockAtArrival_y':'caa_our', 'ApplSeqNum_x':'raw_ApplSeqNum', 'ApplSeqNum_y':'ApplSeqNum', 'secid_x':'secid', 'date_x':'date'})

1127

1127

1127

In [5]:
message1 = message[message['updateType'] == 0]
test = test.sort_values(by=['order', 'caa_our'])
test['ApplSeqNum'] = test.groupby('order')['ApplSeqNum'].transform('first')
test = pd.merge(message1[['secid', 'ApplSeqNum', 'clockAtArrival']], test, on=['secid', 'ApplSeqNum'])
test = test.rename(columns={'clockAtArrival':'caa_triggered'})
trade['secid'] = trade['SecurityID'] + 2000000
test = pd.merge(test, trade[['secid', 'sequenceNo', 'BidApplSeqNum', 'OfferApplSeqNum', 'ApplSeqNum']], on=['secid', 'sequenceNo'], how='left')
display(test[test['updateType'] == 0].shape[0])
test['aggApplSeqNum'] = np.where(test['BidApplSeqNum'] > test['OfferApplSeqNum'], test['BidApplSeqNum'], test['OfferApplSeqNum'])
test = test.rename(columns={'ApplSeqNum_x':'tri_ApplSeqNum', 'ApplSeqNum_y':'trade_ApplSeqNum'})
test['aggApplSeqNum'] = np.where(test['updateType'] == 3, test['trade_ApplSeqNum'], test['aggApplSeqNum'])

1127

In [6]:
mbd['tri_ApplSeqNum'] = mbd['asn']
test = pd.merge(test, mbd[['tri_ApplSeqNum', 'secid', 'caa']], on=['secid', 'tri_ApplSeqNum'], how='left')
test = test[(test['caa_triggered'] == test['caamd']) & (test['caamd'] == test['caa'])]
mbd['aggApplSeqNum'] = mbd['asn']
test = test.rename(columns={'caa':'caamd1'})
test = pd.merge(test, mbd[['aggApplSeqNum', 'secid', 'caa']], on=['secid', 'aggApplSeqNum'], how='left')

In [17]:
test

,secid,tri_ApplSeqNum,caa_triggered,StockSeqNum,caa_raw,updateType,sequenceNo,raw_ApplSeqNum,date,innerSeq,...,start_time,accCode,caa_our,caamd,BidApplSeqNum,OfferApplSeqNum,trade_ApplSeqNum,aggApplSeqNum,caamd1,caa
0,2000001,1103406.0,1609119080446768,1.011035e+08,1609119080450480,0,10118369,1103452.0,20201228,0,...,1.609119e+15,9461.0,1609119080446836,1609119080446768,NaN,NaN,NaN,NaN,1609119080446768,NaN
1,2000001,1103406.0,1609119080446768,1.011035e+08,1609119083735567,3,10311002,1103452.0,20201228,1,...,1.609119e+15,9461.0,1609119083735450,1609119080446768,1103452.0,0.0,1124636.0,1124636.0,1609119080446768,NaN
2,2000001,1184661.0,1609119093496368,1.011847e+08,1609119093499611,0,10837485,1184692.0,20201228,0,...,1.609119e+15,9461.0,1609119093496403,1609119093496368,NaN,NaN,NaN,NaN,1609119093496368,NaN
3,2000001,1184661.0,1609119093496368,1.011847e+08,1609119093928219,4,10856028,1184692.0,20201228,1,...,1.609119e+15,9461.0,1609119093928151,1609119093496368,1184692.0,1186812.0,1186814.0,1186812.0,1609119093496368,1.609119e+15
4,2000001,1194656.0,1609119095389343,1.011947e+08,1609119095391839,0,10930716,1194719.0,20201228,0,...,1.609119e+15,9461.0,1609119095389392,1609119095389343,NaN,NaN,NaN,NaN,1609119095389343,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641,2300630,2023970.0,1609119255105369,3.006300e+13,1609119255109093,4,18580465,2023983.0,20201228,1,...,1.609119e+15,9448.0,1609119255108942,1609119255105369,2023970.0,2023983.0,2023984.0,2023983.0,1609119255105369,1.609119e+15
2642,2300630,2417360.0,1609119337220759,3.006300e+13,1609119337223351,0,22196407,2417366.0,20201228,0,...,1.609119e+15,9448.0,1609119337220797,1609119337220759,NaN,NaN,NaN,NaN,1609119337220759,NaN
2643,2300630,2417360.0,1609119337220759,3.006300e+13,1609119337223355,4,22196408,2417366.0,20201228,1,...,1.609119e+15,9448.0,1609119337223288,1609119337220759,2417366.0,2417360.0,2417367.0,2417366.0,1609119337220759,1.609119e+15
2644,2300630,3274720.0,1609119510562194,3.006300e+13,1609119510565881,0,29933804,3274738.0,20201228,0,...,1.609120e+15,9448.0,1609119510562221,1609119510562194,NaN,NaN,NaN,NaN,1609119510562194,NaN


In [45]:
re = test[(test['updateType'] == 4) & (~test['caa'].isnull())]
(re['caa'] - re['caa_raw']).describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]).astype('int64')

count    1351
mean       -8
std        24
min      -659
10%        -8
20%        -5
30%        -4
40%        -4
50%        -4
60%        -3
70%        -3
80%        -3
90%        -2
95%        -2
99%        -1
max         0
dtype: int64

In [52]:
re[re['caa'] - re['caa_raw'] == 0]

,secid,tri_ApplSeqNum,caa_triggered,StockSeqNum,caa_raw,updateType,sequenceNo,raw_ApplSeqNum,date,innerSeq,...,accCode,caa_our,caamd,BidApplSeqNum,OfferApplSeqNum,trade_ApplSeqNum,aggApplSeqNum,caamd1,caa,OrderType
459,2000860,1454588.0,1609119187386178,8.600145e+10,1609119187389551,4,15523129,1454610.0,20201228,1,...,9461.0,1609119187389516,1609119187386178,1408611.0,1454610.0,1454611.0,1454610.0,1609119187386178,1.609119e+15,2
575,2000960,7501975.0,1609121063238275,9.600750e+10,1609121063241815,4,78012895,7501979.0,20201228,1,...,9448.0,1609121063241891,1609121063238275,7501979.0,7501975.0,7501980.0,7501979.0,1609121063238275,1.609121e+15,2
1502,2002456,3683379.0,1609119668215414,2.456037e+11,1609119668219296,4,35826423,3683398.0,20201228,1,...,9461.0,1609119668219393,1609119668215414,3683379.0,3683398.0,3683399.0,3683398.0,1609119668215414,1.609120e+15,2
1801,2002624,3525968.0,1609119635512729,2.624035e+11,1609119637008256,4,34577141,3525989.0,20201228,3,...,9461.0,1609119637008181,1609119635512729,3525989.0,3531581.0,3531582.0,3531581.0,1609119635512729,1.609120e+15,2
1950,2002831,17480852.0,1609133490663099,2.831175e+11,1609133490666484,4,184340152,17480863.0,20201228,1,...,9448.0,1609133490666580,1609133490663099,17480863.0,17480836.0,17480864.0,17480863.0,1609133490663099,1.609133e+15,2
2092,2002946,3086634.0,1609119568869671,2.946031e+11,1609119568875114,4,31981261,3086648.0,20201228,1,...,9448.0,1609119568875129,1609119568869671,3086648.0,3086634.0,3086649.0,3086648.0,1609119568869671,1.609120e+15,2
2146,2300010,11874917.0,1609122727423058,3.000101e+13,1609122727426660,4,112096642,11874928.0,20201228,1,...,9448.0,1609122727426771,1609122727423058,11874917.0,11874928.0,11874929.0,11874928.0,1609122727423058,1.609123e+15,2
2527,2300413,6920497.0,1609120842848013,3.004131e+13,1609120842851723,4,72090028,6920527.0,20201228,1,...,9461.0,1609120842851721,1609120842848013,6920497.0,6920527.0,6920528.0,6920527.0,1609120842848013,1.609121e+15,2


In [53]:
trade[trade['sequenceNo'] == 15523129]

,clockAtArrival,sequenceNo,TransactTime,SecurityID,ApplSeqNum,cum_volume,ExecType,TradePrice,TradeQty,TradeMoney,BidApplSeqNum,OfferApplSeqNum,secid
3769547,1609119187389551,15523129,93309000,860,1454611,508480,F,692700,1000,692700000,1408611,1454610,2000860


In [54]:
mbd[mbd['asn'] == 1454610]

,ms,cl,tid,caa,sno,snoss,snotr,secid,src,smc,...,bs3,bs2,bs1,as1,as2,as3,as4,as5,tri_ApplSeqNum,aggApplSeqNum
45593,09:33:07.388742,1609119187389603,27792,1609119187389551,15523128,0,0,2000860,100,2,...,200,100,800,800,1600,500,15650,1300,1454610,1454610
42165,09:32:34.458744,1609119154459153,27792,1609119154459118,13943482,0,0,2002128,100,2,...,1200,3100,2000,900,24600,14200,11600,13700,1454610,1454610


In [19]:
re = test[(~test['caa'].isnull()) & (test['updateType'] == 3)]
(re['caa_our'] - re['caa']).describe([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99]).astype('int64')

count      5
mean     -77
std       79
min     -198
10%     -156
20%     -115
30%      -91
40%      -84
50%      -77
60%      -55
70%      -33
80%      -16
90%       -4
95%        1
99%        6
max        8
dtype: int64

In [72]:
re[re['caa_our'] - re['caa'] == 36133328]

,secid,tri_ApplSeqNum,caa_triggered,StockSeqNum,caa_raw,updateType,sequenceNo,raw_ApplSeqNum,date,innerSeq,order,start_time,accCode,caa_our,caamd,BidApplSeqNum,OfferApplSeqNum,aggApplSeqNum,trade_ApplSeqNum,caamd1,caa
2040,2002938,6067083.0,1609120502993695,2.938061e+11,1609120539129872,3,64128789,6067098.0,20201228,2,114502.0,1.609121e+15,9461.0,1609120539129812,1609120502993695,6067098.0,0.0,6067098.0,6173516.0,1609120502993695,1.609121e+15


In [41]:
pd.set_option('max_columns', 200)
mbd[(mbd['asn'] == 1103406) & (mbd['secid'] == 2000001)]

,ms,cl,tid,caa,sno,snoss,snotr,secid,src,smc,asn,tstr,tst,mrmh,cvl,cto,lpr,opx,bp5,bp4,bp3,bp2,bp1,ap1,ap2,ap3,ap4,ap5,bs5,bs4,bs3,bs2,bs1,as1,as2,as3,as4,as5
16205,09:31:20.445750,1609119080446815,27792,1609119080446768,10117962,0,0,2000001,100,2,1103406,93122060,93122060,2344770,1745400,31449887.78,1799,1802,1794,1795,1796,1797,1798,1802,1803,1804,1805,1806,1800,314400,6200,7200,6100,23900,49900,80000,218922,9700


In [62]:
orderLog[orderLog['order'] == 114502][['clockAtArrival', 'caamd', 'ApplSeqNum', 'updateType', 'absOrderSize', 'absFilledThisUpdate', 'tradePrice']]

,clockAtArrival,caamd,ApplSeqNum,updateType,absOrderSize,absFilledThisUpdate,tradePrice
439535,1609120502993738,1609120502993695,6067083.0,0,500,0,-1.00
439536,1609120502996617,1609120502993695,0.0,2,500,0,-1.00
439537,1609120504845980,1609120502993695,0.0,4,500,300,50.71
439538,1609120539127331,1609120502993695,0.0,1,500,0,-1.00
439539,1609120539129812,1609120502993695,0.0,3,500,0,-1.00


In [74]:
test[test['order'] == 114502]

,secid,tri_ApplSeqNum,caa_triggered,StockSeqNum,caa_raw,updateType,sequenceNo,raw_ApplSeqNum,date,innerSeq,order,start_time,accCode,caa_our,caamd,BidApplSeqNum,OfferApplSeqNum,aggApplSeqNum,trade_ApplSeqNum,caamd1,caa
2038,2002938,6067083.0,1609120502993695,2.938061e+11,1609120502996484,0,63035496,6067098.0,20201228,0,114502.0,1.609121e+15,9461.0,1609120502993738,1609120502993695,NaN,NaN,NaN,NaN,1609120502993695,NaN
2039,2002938,6067083.0,1609120502993695,2.938061e+11,1609120504845937,4,63083226,6067098.0,20201228,1,114502.0,1.609121e+15,9461.0,1609120504845980,1609120502993695,6067098.0,6071533.0,6071533.0,6071534.0,1609120502993695,1.609121e+15
2040,2002938,6067083.0,1609120502993695,2.938061e+11,1609120539129872,3,64128789,6067098.0,20201228,2,114502.0,1.609121e+15,9461.0,1609120539129812,1609120502993695,6067098.0,0.0,6067098.0,6173516.0,1609120502993695,1.609121e+15


In [66]:
message[message['ApplSeqNum'] == 6067098]

,StockSeqNum,clockAtArrival,updateType,sequenceNo,secid,ApplSeqNum,date,innerSeq
15730024,2.938061e+11,1609120502996484,0,63035496,2002938,6067098.0,20201228,0
15730025,2.938061e+11,1609120504845937,4,63083226,2002938,6067098.0,20201228,1
15730026,2.938061e+11,1609120539129872,3,64128789,2002938,6067098.0,20201228,2


In [70]:
mbd[mbd['asn'] == 6071534]

,ms,cl,tid,caa,sno,snoss,snotr,secid,src,smc,asn,tstr,tst,mrmh,cvl,cto,lpr,opx,bp5,bp4,bp3,bp2,bp1,ap1,ap2,ap3,ap4,ap5,bs5,bs4,bs3,bs2,bs1,as1,as2,as3,as4,as5,tri_ApplSeqNum,aggApplSeqNum
143614,09:50:56.359658,1609120256360709,27792,1609120256360674,55902511,0,0,2002252,100,2,6071534,95057980,95057980,2228824,3834186,27420845.46,710,718,706,707,708,709,710,711,712,713,714,715,28700,51700,49500,81300,338400,149380,30300,105000,8400,16300,6071534,6071534
